In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from tqdm import tqdm

In [3]:
# === 1. TF-IDF (Stemmed) Benzerliği ===
tfidf_df = pd.read_csv("outputs/tfidf_stemmed_cv.csv")
input_vector = tfidf_df.iloc[3].values.reshape(1, -1)
similarities = cosine_similarity(input_vector, tfidf_df.values)[0]
top_5_indices = np.argsort(similarities)[::-1][1:6]

In [5]:
cv_data = pd.read_csv("data/cv_dataset.csv")
print("=== TF-IDF (Stemmed) Benzer Sonuçlar ===")
for idx in top_5_indices:
    print(f"\nCV #{idx} - Skor: {similarities[idx]:.4f}")
    print(cv_data.iloc[idx]["on_yazi"])

=== TF-IDF (Stemmed) Benzer Sonuçlar ===

CV #2245 - Skor: 1.0000
İlgili alandaki tecrübelerim ve becerilerim doğrultusunda katkı sağlayabileceğime inanıyorum.

CV #591 - Skor: 1.0000
İlgili alandaki tecrübelerim ve becerilerim doğrultusunda katkı sağlayabileceğime inanıyorum.

CV #3122 - Skor: 1.0000
İlgili alandaki tecrübelerim ve becerilerim doğrultusunda katkı sağlayabileceğime inanıyorum.

CV #3662 - Skor: 1.0000
İlgili alandaki tecrübelerim ve becerilerim doğrultusunda katkı sağlayabileceğime inanıyorum.

CV #596 - Skor: 1.0000
İlgili alandaki tecrübelerim ve becerilerim doğrultusunda katkı sağlayabileceğime inanıyorum.


In [7]:
# === 2. TF-IDF (Lemmatized) Benzerliği ===
lemmatized_df = pd.read_csv("data/cv_lemmatized.csv")
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(lemmatized_df["on_yazi"].fillna(""))
tfidf_lemmatized_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
tfidf_lemmatized_df.to_csv("outputs/tfidf_lemmatized_cv.csv", index=False)

In [9]:
input_vector = tfidf_lemmatized_df.iloc[3].values.reshape(1, -1)
similarities = cosine_similarity(input_vector, tfidf_lemmatized_df.values)[0]
top_5_indices = np.argsort(similarities)[::-1][1:6]

In [11]:
print("=== TF-IDF (Lemmatized) Benzer Sonuçlar ===")
for idx in top_5_indices:
    print(f"\nCV #{idx} - Skor: {similarities[idx]:.4f}")
    print(lemmatized_df.iloc[idx]["on_yazi"])

=== TF-IDF (Lemmatized) Benzer Sonuçlar ===

CV #1853 - Skor: 1.0000
i̇lgili alandaki tecrübelerim becerilerim doğrultusunda katkı sağlayabileceğime inanıyorum

CV #3654 - Skor: 1.0000
i̇lgili alandaki tecrübelerim becerilerim doğrultusunda katkı sağlayabileceğime inanıyorum

CV #3655 - Skor: 1.0000
i̇lgili alandaki tecrübelerim becerilerim doğrultusunda katkı sağlayabileceğime inanıyorum

CV #1529 - Skor: 1.0000
i̇lgili alandaki tecrübelerim becerilerim doğrultusunda katkı sağlayabileceğime inanıyorum

CV #4255 - Skor: 1.0000
i̇lgili alandaki tecrübelerim becerilerim doğrultusunda katkı sağlayabileceğime inanıyorum


In [13]:
# === 3. Word2Vec Benzerliği Hesaplama ===
df = pd.read_csv("data/cv_lemmatized.csv")
input_text = df.iloc[3]["on_yazi"]
input_tokens = input_text.split()

In [15]:
def average_vector(words, model):
    vectors = [model.wv[word] for word in words if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

In [17]:
model_dir = "models"
model_files = [f for f in os.listdir(model_dir) if f.endswith(".model") and f.startswith("cv_")]
similarity_results = {}

In [21]:
for model_file in tqdm(model_files):
    model_path = os.path.join(model_dir, model_file)
    model_name = model_file.replace(".model", "")
    try:
        model = Word2Vec.load(model_path)
        input_vec = average_vector(input_tokens, model).reshape(1, -1)
        all_vectors = [average_vector(text.split(), model) for text in df["on_yazi"].fillna("")]
        similarities = cosine_similarity(input_vec, all_vectors)[0]
        top_indices = np.argsort(similarities)[::-1][1:6]
        similarity_results[model_name] = {
            "indices": top_indices.tolist(),
            "scores": similarities[top_indices].tolist()
        }
    except Exception as e:
        print(f"Hata oluştu: {model_file} — {e}")

100%|██████████| 16/16 [00:02<00:00,  6.45it/s]


In [23]:
cv_df = pd.read_csv("data/cv_lemmatized.csv")
model_names = list(similarity_results.keys())[:3]

In [25]:
evaluation_data = []
for model_name in model_names:
    indices = similarity_results[model_name]["indices"]
    scores = similarity_results[model_name]["scores"]
    for rank, (idx, score) in enumerate(zip(indices, scores), 1):
        evaluation_data.append({
            "Model": model_name,
            "Sıra": rank,
            "CV_Index": idx,
            "Benzerlik_Skoru": round(score, 4),
            "On_Yazı": cv_df.iloc[idx]["on_yazi"]
        })

evaluation_df = pd.DataFrame(evaluation_data)

In [27]:
# === 4. Jaccard Benzerlik Matrisi ===
def jaccard_similarity(set1, set2):
    intersection = len(set(set1).intersection(set2))
    union = len(set(set1).union(set2))
    return intersection / union if union > 0 else 0

In [29]:
model_names = list(similarity_results.keys())
jaccard_matrix = np.zeros((len(model_names), len(model_names)))

for i, model_a in enumerate(model_names):
    for j, model_b in enumerate(model_names):
        indices_a = set(similarity_results[model_a]["indices"])
        indices_b = set(similarity_results[model_b]["indices"])
        jaccard_matrix[i, j] = jaccard_similarity(indices_a, indices_b)

jaccard_df = pd.DataFrame(jaccard_matrix, index=model_names, columns=model_names)

In [31]:
# === 5. Anlamsal Skor ===
evaluation_df["Anlamsal_Skor"] = [4, 4, 3, 5, 4,
                                  2, 3, 2, 3, 1,
                                  3, 3, 4, 4, 3]  

In [33]:
# === 6. Ortalama Anlamsal Skor ===
ortalama_df = (
    evaluation_df
    .groupby("Model")["Anlamsal_Skor"]
    .mean()
    .reset_index()
    .rename(columns={"Anlamsal_Skor": "Ortalama_Anlamsal_Skor"})
    .sort_values(by="Ortalama_Anlamsal_Skor", ascending=False)
)

In [49]:
# Ortalama değerlendirme tablosunu görselleştir
print("Anlamsal Değerlendirme Tablosu")
print(ortalama_df)

Anlamsal Değerlendirme Tablosu
                                     Model  Ortalama_Anlamsal_Skor
0  cv_lemmatized_model_cbow_window2_dim100                     4.0
2  cv_lemmatized_model_cbow_window4_dim100                     3.4
1  cv_lemmatized_model_cbow_window2_dim300                     2.2


In [51]:
from tabulate import tabulate
print(tabulate(ortalama_df, headers='keys', tablefmt='grid'))

+----+-----------------------------------------+--------------------------+
|    | Model                                   |   Ortalama_Anlamsal_Skor |
+====+=========================================+==========================+
|  0 | cv_lemmatized_model_cbow_window2_dim100 |                      4   |
+----+-----------------------------------------+--------------------------+
|  2 | cv_lemmatized_model_cbow_window4_dim100 |                      3.4 |
+----+-----------------------------------------+--------------------------+
|  1 | cv_lemmatized_model_cbow_window2_dim300 |                      2.2 |
+----+-----------------------------------------+--------------------------+


In [55]:
# Anlamsal değerlendirme tablosunu görsel olarak göster
from IPython.display import display, HTML

# Ortalama skorları göster
ortalama_df = (
    evaluation_df
    .groupby("Model")["Anlamsal_Skor"]
    .mean()
    .reset_index()
    .rename(columns={"Anlamsal_Skor": "Ortalama_Anlamsal_Skor"})
    .sort_values(by="Ortalama_Anlamsal_Skor", ascending=False)
)

# Tabloyu görselleştir
display(HTML("<h3>🔍 Anlamsal Değerlendirme Ortalamaları</h3>"))
display(ortalama_df.style.set_caption("Model Başına Ortalama Anlamsal Skor").background_gradient(cmap='Blues'))


,Model,Ortalama_Anlamsal_Skor
0,cv_lemmatized_model_cbow_window2_dim100,4.000000
2,cv_lemmatized_model_cbow_window4_dim100,3.400000
1,cv_lemmatized_model_cbow_window2_dim300,2.200000


In [57]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)

# Tüm tabloyu stil ile göster
display(HTML("<h3>📋 Model Bazlı İlk 5 Sonuç ve Anlamsal Puanlar</h3>"))
display(evaluation_df.style.set_caption("Her Model İçin İlk 5 Benzer CV ve Puanları").background_gradient(cmap='YlGnBu'))


,Model,Sıra,CV_Index,Benzerlik_Skoru,On_Yazı,Anlamsal_Skor
0,cv_lemmatized_model_cbow_window2_dim100,1,3393,1.000000,i̇lgili alandaki tecrübelerim becerilerim doğrultusunda katkı sağlayabileceğime inanıyorum,4
1,cv_lemmatized_model_cbow_window2_dim100,2,2560,1.000000,i̇lgili alandaki tecrübelerim becerilerim doğrultusunda katkı sağlayabileceğime inanıyorum,4
2,cv_lemmatized_model_cbow_window2_dim100,3,2561,1.000000,i̇lgili alandaki tecrübelerim becerilerim doğrultusunda katkı sağlayabileceğime inanıyorum,3
3,cv_lemmatized_model_cbow_window2_dim100,4,3882,1.000000,i̇lgili alandaki tecrübelerim becerilerim doğrultusunda katkı sağlayabileceğime inanıyorum,5
4,cv_lemmatized_model_cbow_window2_dim100,5,2566,1.000000,i̇lgili alandaki tecrübelerim becerilerim doğrultusunda katkı sağlayabileceğime inanıyorum,4
5,cv_lemmatized_model_cbow_window2_dim300,1,2814,1.000000,i̇lgili alandaki tecrübelerim becerilerim doğrultusunda katkı sağlayabileceğime inanıyorum,2
6,cv_lemmatized_model_cbow_window2_dim300,2,4230,1.000000,i̇lgili alandaki tecrübelerim becerilerim doğrultusunda katkı sağlayabileceğime inanıyorum,3
7,cv_lemmatized_model_cbow_window2_dim300,3,4229,1.000000,i̇lgili alandaki tecrübelerim becerilerim doğrultusunda katkı sağlayabileceğime inanıyorum,2
8,cv_lemmatized_model_cbow_window2_dim300,4,2779,1.000000,i̇lgili alandaki tecrübelerim becerilerim doğrultusunda katkı sağlayabileceğime inanıyorum,3
9,cv_lemmatized_model_cbow_window2_dim300,5,312,1.000000,i̇lgili alandaki tecrübelerim becerilerim doğrultusunda katkı sağlayabileceğime inanıyorum,1
